Time step: 1


MethodError: MethodError: no method matching getindex(::Float64, ::BitVector)
The function `getindex` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  getindex(::Number, !Matched::Integer)
   @ Base number.jl:96
  getindex(::Number)
   @ Base number.jl:95
  getindex(::Number, !Matched::Integer...)
   @ Base number.jl:101
  ...
